# Working With PDF

## 1. Module import

In [64]:
import pypdf
from pypdf import PdfReader
import pdfplumber
import fitz
from pathlib import Path

## 2. Handling data

In [65]:
pdf_path = '../data_import/pdf_mock_file.pdf'
target_path = Path('../data_export/file_pdf')

In [66]:
reader = PdfReader(pdf_path)
for  i in range(len(reader.pages)):
    page = reader.pages[i]

In [67]:
target_text_path = target_path / "text"
target_text_path.mkdir(parents=True, exist_ok=True)

doc = fitz.open(pdf_path)

all_text = ""

for page in doc:
    all_text += page.get_text()

output_text_path = target_text_path / "extracted_text.txt"

with open(output_text_path, 'w') as file:
    file.write(all_text)

doc.close()

print(f"Extracted text has been saved to {output_file_path}")

Extracted text has been saved to ../data_export/file_pdf/text/extracted_text.txt


In [68]:
target_images_path = target_path / "images"
target_images_path.mkdir(parents=True, exist_ok=True)

for i, page in enumerate(reader.pages):
    for image_index, img in enumerate(page.images):
        output_images_path = target_images_path / f"page_{i}_image_{image_index}.jpg"
        with open(output_images_path, "wb") as f:
            f.write(img.data)
print(f"Extracted images has been saved to {output_images_path}")

Extracted images has been saved to ../data_export/file_pdf/images/page_2_image_5.jpg
